In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.raw_contract import RawContract, download_s3_object
from core.dataset_contract import DatasetContract

session = SessionHelper().session
db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
        delimiter: str = db_transform.variables.delimiter # The input file delimiter
        skip_rows: int = db_transform.variables.skip_rows # the number of rows to skip before reading
        encoding: str = db_transform.variables.encoding # The file encoding
        input_file_prefix: str = db_transform.variables.input_file_prefix # The prefix of input files to read in

## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead
transform = Transform()
transform.delimiter = ", "
transform.skip_rows = 0
transform.encoding = "utf8"
transform.input_file_prefix = "BRIOVA_"
transform.dataset_name = "init_ingest_patient_status"

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This transformation takes a collection of raw files based on location and their prefix, then creates a dataset from the data and outputs it to S3. The delimiter and encodings can be set, as well as a number of rows at the top to be skipped.

### Transformation

In [ ]:
### Setup important imports
import pandas as pd
from pandas.errors import EmptyDataError
import os
import tempfile
from os import path
from encodings.aliases import aliases
from core.logging import get_logger

In [ ]:
### Setup for transformation object
transform = Transform()
logger = get_logger(f"core.transforms.{transform.state}.{transform.name}")
final_dataframe = pd.DataFrame()

In [ ]:
transform.publish_contract.dataset = transform.name + "/" + transform.input_file_prefix

input_contract = RawContract(branch=BRANCH_NAME,
                            parent=transform.pharmaceutical_company,
                            child=transform.brand)

logger.debug(f"Ingesting files from path : {input_contract.s3_path}")

In [ ]:
def ingest_file(f, filename, input_contract):
    input_s3_url = input_contract.s3_path_with_filename(filename)
    logger.debug(f"Ingesting file: {filename}")
    df = pd.read_csv(f, dtype="str", encoding=transform.encoding, compression="infer", sep=transform.delimiter, skiprows=transform.skip_rows)
    return df

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe
valid_encodings = set(aliases.keys()) | set(aliases.values())
if transform.encoding not in valid_encodings:
    invalid_encoding_message = f"initial_ingest_configuration {transform.id} has invalid encoding: {transform.encoding}"
    logger.critical(invalid_encoding_message)
    raise ValueError(invalid_encoding_message)
if len(transform.delimiter) < 1:
    invalid_delimiter_message = f"initial_ingest_configuration {transform.id} has no delimiter"
    logger.critical(invalid_delimiter_message)
    raise ValueError(invalid_delimiter_message)

in_files = input_contract.list_files(transform.input_file_prefix)

for f in in_files:
    filename = path.basename(f)
    with input_contract.download_raw_file(filename) as fn:
        try:
            df = ingest_file(fn, filename, input_contract)
            final_dataframe = final_dataframe.append(df)
        except EmptyDataError as e:
            pass

In [ ]:
## That's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session, False)
session.close()